In [1]:
import tensorflow as tf
import string
import requests
import pandas as pd

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout

## Load Dataset

In [2]:
df = pd.read_csv('Datasets/Spanish-Story-Dataset.txt', delimiter = "\n", header = None)

In [3]:
Text = df[0].tolist()

In [4]:
len(Text)

2040

In [5]:
len(" ".join(Text))

118900

## Build LSTM Model and Prepare X and y

In [6]:
token = Tokenizer()
token.fit_on_texts(Text)

In [7]:
token.word_index

{'el': 1,
 'y': 2,
 'a': 3,
 'que': 4,
 'se': 5,
 'la': 6,
 'de': 7,
 'no': 8,
 'le': 9,
 'su': 10,
 'lo': 11,
 'en': 12,
 'un': 13,
 'los': 14,
 'al': 15,
 'me': 16,
 'ya': 17,
 'dice': 18,
 'una': 19,
 'por': 20,
 'con': 21,
 'te': 22,
 'para': 23,
 'coyote': 24,
 'cuando': 25,
 'como': 26,
 'señor': 27,
 'hombre': 28,
 'muy': 29,
 'así': 30,
 'voy': 31,
 'casa': 32,
 'pero': 33,
 'qué': 34,
 'está': 35,
 'les': 36,
 'yo': 37,
 'es': 38,
 'dijo': 39,
 'las': 40,
 'estaba': 41,
 'aquí': 42,
 'donde': 43,
 'bien': 44,
 'si': 45,
 'ahí': 46,
 'ahora': 47,
 'dinero': 48,
 'sí': 49,
 'fue': 50,
 'va': 51,
 'había': 52,
 'del': 53,
 'comer': 54,
 'muchacho': 55,
 'más': 56,
 'tlacuache': 57,
 'todo': 58,
 'día': 59,
 'sólo': 60,
 'tenía': 61,
 'sus': 62,
 'esposa': 63,
 'vez': 64,
 'mucho': 65,
 'este': 66,
 'ver': 67,
 'mi': 68,
 '¿qué': 69,
 'él': 70,
 'boa': 71,
 'todos': 72,
 'hijo': 73,
 'porque': 74,
 'van': 75,
 'vamos': 76,
 'eso': 77,
 'hasta': 78,
 'nada': 79,
 'animales': 80,
 '

In [8]:
encoded_text = token.texts_to_sequences(Text)

In [9]:
encoded_text

[[759, 336, 760, 4, 12, 1, 211, 761, 1501],
 [1502, 337, 1503, 2, 4, 338, 7, 762, 212, 388, 1504],
 [4, 1505, 1, 999, 763, 7, 13, 28],
 [87, 5, 18, 4, 338, 212, 29, 1506, 2, 4, 20, 619, 5, 445],
 [1507, 58, 1, 178, 198, 389, 2, 25, 620, 213],
 [1508, 20, 6, 514, 19, 1000, 7, 1001, 4, 5, 1002, 20],
 [390, 1509, 20, 66, 263, 1510, 81, 1511, 3, 14, 446, 1003, 187],
 [1512, 4, 5, 515, 21, 92, 1513, 2, 5, 14, 1514],
 [93, 621, 16, 6, 622, 68, 1515, 2, 1516, 96, 764, 30, 1517],
 [18, 4, 94, 391, 171, 13, 1518, 7, 1519, 3, 96, 2, 3, 10, 73],
 [36, 392, 1, 199, 339],
 [92, 59, 227, 73, 5, 623, 291, 37, 41, 1520, 5, 16, 292, 2, 16, 39],
 [1004, 4, 17, 264, 315, 7, 765, 624, 4, 1005, 66, 393, 4],
 [1006],
 [37, 9, 1007],
 [1008, 516, 1521, 766, 316, 624, 69, 50, 11, 4, 516],
 [70, 16, 265],
 [1522, 4, 1523, 3, 214, 15, 211, 2, 12, 1, 113, 16, 1009, 21],
 [19, 120, 1010, 7, 1524, 13, 1525, 53, 340, 394, 4, 1011],
 [10, 200, 83, 29, 1012, 61, 1, 767, 1013, 1526, 2],
 [16, 239, 4, 3, 240, 88, 9, 15

In [10]:
vocab_size = len(token.word_counts) + 1

In [11]:
print(vocab_size)

3372


## Prepare Training Data

In [12]:
datalist = []
for d in encoded_text:
    if len(d) > 1:#skip 1 len sentence 
        for i in range(2, len(d)):
            datalist.append(d[:i])
            print(d[:i])

[759, 336]
[759, 336, 760]
[759, 336, 760, 4]
[759, 336, 760, 4, 12]
[759, 336, 760, 4, 12, 1]
[759, 336, 760, 4, 12, 1, 211]
[759, 336, 760, 4, 12, 1, 211, 761]
[1502, 337]
[1502, 337, 1503]
[1502, 337, 1503, 2]
[1502, 337, 1503, 2, 4]
[1502, 337, 1503, 2, 4, 338]
[1502, 337, 1503, 2, 4, 338, 7]
[1502, 337, 1503, 2, 4, 338, 7, 762]
[1502, 337, 1503, 2, 4, 338, 7, 762, 212]
[1502, 337, 1503, 2, 4, 338, 7, 762, 212, 388]
[4, 1505]
[4, 1505, 1]
[4, 1505, 1, 999]
[4, 1505, 1, 999, 763]
[4, 1505, 1, 999, 763, 7]
[4, 1505, 1, 999, 763, 7, 13]
[87, 5]
[87, 5, 18]
[87, 5, 18, 4]
[87, 5, 18, 4, 338]
[87, 5, 18, 4, 338, 212]
[87, 5, 18, 4, 338, 212, 29]
[87, 5, 18, 4, 338, 212, 29, 1506]
[87, 5, 18, 4, 338, 212, 29, 1506, 2]
[87, 5, 18, 4, 338, 212, 29, 1506, 2, 4]
[87, 5, 18, 4, 338, 212, 29, 1506, 2, 4, 20]
[87, 5, 18, 4, 338, 212, 29, 1506, 2, 4, 20, 619]
[87, 5, 18, 4, 338, 212, 29, 1506, 2, 4, 20, 619, 5]
[1507, 58]
[1507, 58, 1]
[1507, 58, 1, 178]
[1507, 58, 1, 178, 198]
[1507, 58, 1, 178

## Padding

In [13]:
max_length = 20
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')
sequences

array([[  0,   0,   0, ...,   0, 759, 336],
       [  0,   0,   0, ..., 759, 336, 760],
       [  0,   0,   0, ..., 336, 760,   4],
       ...,
       [  0,   0,   0, ...,  76, 432,   3],
       [  0,   0,   0, ...,   0,  23,   4],
       [  0,   0,   0, ...,  23,   4,  86]], dtype=int32)

In [14]:
X = sequences[:, :-1]

In [15]:
y = sequences[:, -1]

In [16]:
y = to_categorical(y, num_classes=vocab_size)

In [17]:
y #one hot encoder

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
seq_length = X.shape[1]

In [19]:
seq_length

19

## LSTM Model Training

In [56]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [57]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 50)            168600    
_________________________________________________________________
lstm (LSTM)                  (None, 19, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 3372)              340572    
Total params: 660,072
Trainable params: 660,072
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [59]:
model.fit(X, y, batch_size=32, epochs=100)


Epoch 1/100
567/567 [==============================] - 25s 34ms/step - loss: 6.7207 - accuracy: 0.0420
Epoch 2/100
567/567 [==============================] - 19s 33ms/step - loss: 5.9980 - accuracy: 0.0427
Epoch 3/100
567/567 [==============================] - 19s 33ms/step - loss: 5.7644 - accuracy: 0.0573
Epoch 4/100
567/567 [==============================] - 18s 33ms/step - loss: 5.6124 - accuracy: 0.0670
Epoch 5/100
567/567 [==============================] - 19s 33ms/step - loss: 5.5025 - accuracy: 0.0775
Epoch 6/100
567/567 [==============================] - 19s 33ms/step - loss: 5.3916 - accuracy: 0.0836
Epoch 7/100
567/567 [==============================] - 20s 35ms/step - loss: 5.2590 - accuracy: 0.0925
Epoch 8/100
567/567 [==============================] - 20s 35ms/step - loss: 5.2019 - accuracy: 0.0961
Epoch 9/100
567/567 [==============================] - 19s 33ms/step - loss: 5.1015 - accuracy: 0.1050
Epoch 10/100
567/567 [==============================] - 19s 33ms/step - l

In [60]:
poetry_length = 20

def generate_poetry(seed_text, n_lines):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

      y_pred = np.argmax(model.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)




In [61]:
seed_text = 'Había una '
generate_poetry(seed_text, 50)

vez un ratón que otros insectos por regresar a ver el muchacho le dice a su esposa y va cantar
y no tienen nada en pleno se quedó dormido sobre el camino y grande fuerte y se puso a reír
el señor dice mesa pon el mantel y sin elotes en el cerro y se va la señora se sentó
menos me voy a morder te voy a comer y le dice el hombre dice ¡pobre coyote empezaron y mañana
lo que le gustaba andar engañando a sus hermosas grandes pasaba naranjas que él se asustó y luego se puso
a andar mañana a la mañana siguiente muy lejos la boa le dice ahora sí te voy a mucha engañando
el hombre dice ahorita lo que se va a pasar a los demás porque ya no te di y el
hombre le dice a su esposa y se va a pasar la señora lleva todo lo que se trataba y
le dice el hombre dice ahorita lo que se va a pasar la señora lleva todo lo que se vendía
se adelantó y lo levantó como adultos ruidos le dice el hombre dice ahorita lo que digo y y de
inmediato se va a traer la circunstancia del señor y lo hace el tlacuache le amarr

In [20]:
model2 = Sequential()
model2.add(Embedding(vocab_size, 50, input_length=seq_length))
model2.add(LSTM(256, return_sequences=True))
model2.add(Dropout(0.1))
model2.add(LSTM(512,return_sequences=True))
model2.add(Dropout(0.1))
model2.add(LSTM(1024,return_sequences=True))
model2.add(Dropout(0.2))
model2.add(LSTM(512))
model2.add(Dense(512, activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(vocab_size, activation='softmax'))

In [21]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 50)            168600    
_________________________________________________________________
lstm (LSTM)                  (None, 19, 256)           314368    
_________________________________________________________________
dropout (Dropout)            (None, 19, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 19, 512)           1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 19, 1024)          6295552   
_________________________________________________________________
dropout_2 (Dropout)          (None, 19, 1024)          0

In [23]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model2.fit(X, y, batch_size=32, epochs=100)


Epoch 1/100
567/567 [==============================] - 37s 48ms/step - loss: 6.6767 - accuracy: 0.0340
Epoch 2/100
567/567 [==============================] - 27s 48ms/step - loss: 6.1890 - accuracy: 0.0368
Epoch 3/100
567/567 [==============================] - 27s 48ms/step - loss: 6.1737 - accuracy: 0.0374
Epoch 4/100
567/567 [==============================] - 27s 48ms/step - loss: 6.1284 - accuracy: 0.0355
Epoch 5/100
567/567 [==============================] - 27s 48ms/step - loss: 6.1602 - accuracy: 0.0380
Epoch 6/100
567/567 [==============================] - 28s 49ms/step - loss: 6.1554 - accuracy: 0.0417
Epoch 7/100
567/567 [==============================] - 28s 49ms/step - loss: 6.1510 - accuracy: 0.0411
Epoch 8/100
567/567 [==============================] - 28s 49ms/step - loss: 6.1569 - accuracy: 0.0394
Epoch 9/100
567/567 [==============================] - 28s 49ms/step - loss: 6.1594 - accuracy: 0.0389
Epoch 10/100
567/567 [==============================] - 28s 49ms/step - l

In [27]:
poetry_length = 20

def generate_poetry(seed_text, n_lines):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

      y_pred = np.argmax(model2.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)




In [28]:
seed_text = 'Había una '
generate_poetry(seed_text, 50)

y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y
y y y y y y y y y y y y y y y y y y y y


In [20]:
model3 = Sequential()
model3.add(Embedding(vocab_size, 50, input_length=seq_length))
model3.add(LSTM(256, return_sequences=True))
model3.add(Dropout(0.2))
model3.add(LSTM(256))
model3.add(Dropout(0.2))
model3.add(Dense(vocab_size, activation='softmax'))

In [21]:
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 50)            168600    
_________________________________________________________________
lstm (LSTM)                  (None, 19, 256)           314368    
_________________________________________________________________
dropout (Dropout)            (None, 19, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 3372)              866604    
Total params: 1,874,884
Trainable params: 1,874,884
Non-trainable params: 0
______________________________________________

In [22]:
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model3.fit(X, y, batch_size=32, epochs=210)


Epoch 1/210
567/567 [==============================] - 25s 18ms/step - loss: 6.6174 - accuracy: 0.0344
Epoch 2/210
567/567 [==============================] - 10s 17ms/step - loss: 6.0510 - accuracy: 0.0504
Epoch 3/210
567/567 [==============================] - 10s 17ms/step - loss: 5.8359 - accuracy: 0.0678
Epoch 4/210
567/567 [==============================] - 10s 17ms/step - loss: 5.7419 - accuracy: 0.0722
Epoch 5/210
567/567 [==============================] - 10s 18ms/step - loss: 5.5925 - accuracy: 0.0767
Epoch 6/210
567/567 [==============================] - 10s 17ms/step - loss: 5.4310 - accuracy: 0.0877
Epoch 7/210
567/567 [==============================] - 10s 18ms/step - loss: 5.3553 - accuracy: 0.0961
Epoch 8/210
567/567 [==============================] - 10s 17ms/step - loss: 5.2100 - accuracy: 0.1053
Epoch 9/210
567/567 [==============================] - 10s 17ms/step - loss: 5.0907 - accuracy: 0.1109
Epoch 10/210
567/567 [==============================] - 10s 17ms/step - l

In [24]:
poetry_length = 20

def generate_poetry(seed_text, n_lines):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

      y_pred = np.argmax(model3.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)




In [25]:
seed_text = 'Había una '
generate_poetry(seed_text, 50)

vez un señor que recibió el encargo de hacer una cueva que se columpiaba en una piedra lastimándose animal comunidad
formado para la vida los quiere mucho su leña y fueron muy lejos y por ello necesito llevar suficiente usted
el mantel pero no lo diga el señor escogió el palo al suelo le dice palo dale hermano vio hermano
fuertemente para que ya no le vendieran nada comenzó a pelar una tuna y me la trae y ya no
lejos de ahí estaba una barranca y dijo el coyote se espina cuando vieron un lugar y cada quien le
dice el burro que alimentaste y cuidaste es tuyo llévatelo no tengo mucho dinero en el cerro porque mucha guardaba
en el bosque el hermano encontró la ollita en monedas de oro el terminar de señor y le comunidad vestida
con ropas y les pone una vara de chinamite a sus animales no tardó mucho el muchacho no ya hermano
fuertemente para que ya no le vendieran nada comenzó a pelar una tuna y me la trae y ya no
lejos de ahí estaba una barranca y dijo el coyote se espina cuando vieron un

In [36]:
poetry_length = 20

def generate_poetry(seed_text, n_lines):
    final_text = seed_text
    for i in range(n_lines):
        text = []
        for _ in range(poetry_length):
          encoded = token.texts_to_sequences([seed_text])
          encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

          y_pred = np.argmax(model3.predict(encoded), axis=-1)

          predicted_word = ""
          for word, index in token.word_index.items():
            if index == y_pred:
              predicted_word = word
              break

          seed_text = seed_text + ' ' + predicted_word
          text.append(predicted_word)

        seed_text = text[-1]
        text = ' '.join(text)
        final_text = final_text + text 
    return final_text
    

In [37]:
seed_text = 'Había una '
last_text = generate_poetry(seed_text, 5)
print(last_text)

Había una vez un señor que recibió el encargo de hacer una cueva que se columpiaba en una piedra lastimándose animal comunidadformado para la vida los quiere mucho su leña y fueron muy lejos y por ello necesito llevar suficiente ustedel mantel pero no lo diga el señor escogió el palo al suelo le dice palo dale hermano vio hermanofuertemente para que ya no le vendieran nada comenzó a pelar una tuna y me la trae y ya nolejos de ahí estaba una barranca y dijo el coyote se espina cuando vieron un lugar y cada quien le


In [45]:
print(last_text[:len(last_text)-10])

ad vestidacon ropas y les pone una vara de chinamite a sus animales no tardó mucho el muchacho no ya hermanofuertemente para que ya no le vendieran nada comenzó a pelar una tuna y me la trae y ya nolejos de ahí estaba una barranca y dijo el coyote se espina cuando vieron un lugar y cada quien ledice el burro que alimentaste y cuidaste es tuyo llévatelo no tengo mucho dinero en el cerro porque mucha guardabaen el bosque el hermano encontró la ollita en monedas de oro el terminar de señor y le comunid


In [49]:
seed_text = 'Había una '
final_text = seed_text
for i in range(10):
    last_text = generate_poetry(seed_text, 5)
    #final_text + final_text + last_text[:len(last_text)-10]
    print(last_text[:len(last_text)-10])
    seed_text = last_text[len(last_text)-10:]
    #print("SEED",seed_text)
    
print(final_text)

Había una vez un señor que recibió el encargo de hacer una cueva que se columpiaba en una piedra lastimándose animal comunidadformado para la vida los quiere mucho su leña y fueron muy lejos y por ello necesito llevar suficiente ustedel mantel pero no lo diga el señor escogió el palo al suelo le dice palo dale hermano vio hermanofuertemente para que ya no le vendieran nada comenzó a pelar una tuna y me la trae y ya nolejos de ahí estaba una barranca y dijo el coyote se espina cuando vieron un lugar y cad
a quien ledice su mujer tú apúrate el hombre perezoso no trabajaba y por eso no se voy hermano te la víborale responde frente a ti camina despacio hacia el camino está ahí se podían sobre esta ocasión se les vezque las piedras eran lanzadas solamente rebotaban en las tiendas tomates panes chiles naranjas todo lo que hace la viejitaespera espera un poco ahorita regreso la viejita necesito algo de leña para suficiente una mesa se sentó y otrasveces se puede ver un mercado o una casa en l

In [51]:
seed_text = 'Había una '
last_text = generate_poetry(seed_text, 5)
print(last_text)

Había una vez un señor que recibió el encargo de hacer una cueva que se columpiaba en una piedra lastimándose animal comunidadformado para la vida los quiere mucho su leña y fueron muy lejos y por ello necesito llevar suficiente ustedel mantel pero no lo diga el señor escogió el palo al suelo le dice palo dale hermano vio hermanofuertemente para que ya no le vendieran nada comenzó a pelar una tuna y me la trae y ya nolejos de ahí estaba una barranca y dijo el coyote se espina cuando vieron un lugar y cada quien le


In [52]:
seed_text = 'quien le '
last_text = generate_poetry(seed_text, 5)
print(last_text)

quien le habla por más que busque no ve quien es el más poderoso pelearemos y veremos quien es el camino hermanofuertemente para que ya no le vendieran nada comenzó a pelar una tuna y me la trae y ya nolejos de ahí estaba una barranca y dijo el coyote se espina cuando vieron un lugar y cada quien ledice el burro que alimentaste y cuidaste es tuyo llévatelo no tengo mucho dinero en el cerro porque mucha guardabaen el bosque el hermano encontró la ollita en monedas de oro el terminar de señor y le comunidad vestida


In [53]:
seed_text = 'vestida '
last_text = generate_poetry(seed_text, 5)
print(last_text)

vestida con ropas y les pone una vara de chinamite a sus animales no tardó mucho el muchacho no ya hermanofuertemente para que ya no le vendieran nada comenzó a pelar una tuna y me la trae y ya nolejos de ahí estaba una barranca y dijo el coyote se espina cuando vieron un lugar y cada quien ledice el burro que alimentaste y cuidaste es tuyo llévatelo no tengo mucho dinero en el cerro porque mucha guardabaen el bosque el hermano encontró la ollita en monedas de oro el terminar de señor y le comunidad vestida


## i got a eternal loop again

## Second try

In [16]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import os
import datetime
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("TensorFlow version: ", tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5043870494077151883
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7837856576
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9228061813363355125
physical_device_desc: "device: 0, name: Quadro M4000, pci bus id: 0000:00:05.0, compute capability: 5.2"
]
TensorFlow version:  2.4.1


In [18]:
import os
path = os.getcwd()
text = open('Datasets/Spanish-Story-Dataset.txt',  "rb").read().decode(encoding='utf-8')
print("Text is {} characters long".format(len(text)))

Text is 120943 characters long


In [19]:
words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print("Text is {} words long".format(len(words)))

Text is 20254 words long


In [20]:
print(text[:100])


Cuentan nuestros abuelos, que en el cerro llamado Tlacuiloltecatl
anidan grandes serpientes, y que 


In [21]:
vocab = sorted(set(text))
print ('There are {} unique characters'.format(len(vocab)))
char2int = {c:i for i, c in enumerate(vocab)}
int2char = np.array(vocab)
print('Vector:\n')
for char,_ in zip(char2int, range(len(vocab))):
    print(' {:4s}: {:3d},'.format(repr(char), char2int[char]))

There are 80 unique characters
Vector:

 '\n':   0,
 '\r':   1,
 ' ' :   2,
 '!' :   3,
 '(' :   4,
 ')' :   5,
 ',' :   6,
 '.' :   7,
 '1' :   8,
 '2' :   9,
 '4' :  10,
 '5' :  11,
 ':' :  12,
 ';' :  13,
 '?' :  14,
 'A' :  15,
 'B' :  16,
 'C' :  17,
 'D' :  18,
 'E' :  19,
 'F' :  20,
 'G' :  21,
 'H' :  22,
 'I' :  23,
 'J' :  24,
 'L' :  25,
 'M' :  26,
 'N' :  27,
 'O' :  28,
 'P' :  29,
 'Q' :  30,
 'R' :  31,
 'S' :  32,
 'T' :  33,
 'U' :  34,
 'V' :  35,
 'X' :  36,
 'Y' :  37,
 'Z' :  38,
 'a' :  39,
 'b' :  40,
 'c' :  41,
 'd' :  42,
 'e' :  43,
 'f' :  44,
 'g' :  45,
 'h' :  46,
 'i' :  47,
 'j' :  48,
 'l' :  49,
 'm' :  50,
 'n' :  51,
 'o' :  52,
 'p' :  53,
 'q' :  54,
 'r' :  55,
 's' :  56,
 't' :  57,
 'u' :  58,
 'v' :  59,
 'x' :  60,
 'y' :  61,
 'z' :  62,
 '¡' :  63,
 '¿' :  64,
 'É' :  65,
 'Ó' :  66,
 'á' :  67,
 'é' :  68,
 'ì' :  69,
 'í' :  70,
 'ñ' :  71,
 'ò' :  72,
 'ó' :  73,
 'ú' :  74,
 'ü' :  75,
 '—' :  76,
 '“' :  77,
 '”' :  78,
 '…' :  79,


In [22]:
text_as_int = np.array([char2int[ch] for ch in text], dtype=np.int32)
print ('{}\n mapped to integers:\n {}'.format(repr(text[:100]), text_as_int[:100]))

'Cuentan nuestros abuelos, que en el cerro llamado Tlacuiloltecatl\r\nanidan grandes serpientes, y que '
 mapped to integers:
 [17 58 43 51 57 39 51  2 51 58 43 56 57 55 52 56  2 39 40 58 43 49 52 56
  6  2 54 58 43  2 43 51  2 43 49  2 41 43 55 55 52  2 49 49 39 50 39 42
 52  2 33 49 39 41 58 47 49 52 49 57 43 41 39 57 49  1  0 39 51 47 42 39
 51  2 45 55 39 51 42 43 56  2 56 43 55 53 47 43 51 57 43 56  6  2 61  2
 54 58 43  2]


In [24]:
tr_text = text_as_int[:16203] 
val_text = text_as_int[16203:] 
print(text_as_int.shape, tr_text.shape, val_text.shape)

(120943,) (16203,) (104740,)


In [25]:
batch_size = 64
buffer_size = 10000
embedding_dim = 256
epochs = 70
seq_length = 200
examples_per_epoch = len(text)//seq_length
#lr = 0.001 #will use default for Adam optimizer
rnn_units = 1024
vocab_size = len(vocab)

In [26]:
tr_char_dataset = tf.data.Dataset.from_tensor_slices(tr_text)
val_char_dataset = tf.data.Dataset.from_tensor_slices(val_text)
tr_sequences = tr_char_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
tr_dataset = tr_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
val_dataset = val_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(tr_dataset, val_dataset)

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int32, tf.int32)> <BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int32, tf.int32)>


In [31]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2), 
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


In [32]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=batch_size)

In [33]:
for input_example_batch, target_example_batch in tr_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "respectively: batch_size, sequence_length, vocab_size")

(64, 200, 80) respectively: batch_size, sequence_length, vocab_size


In [34]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           20480     
_________________________________________________________________
dropout (Dropout)            (64, None, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dropout_1 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dropout_2 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
dense_1 (Dense)              (64, None, 80)           

Untrained model output:



In [35]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print("Input: \n", repr("".join(int2char[input_example_batch[0]])))
print()
print("Predictions: \n", repr("".join(int2char[sampled_indices ])))

Input: 
 ' pensar y actuar, además convencer a otros para que cuiden a la\r\nmadre tierra. Llegó hasta la cercanía de la casa de la niña y la bajó\r\nsuavemente, ella corrió a su casa, vió a su mamá que lloraba y l'

Predictions: 
 'rv!2Uó—áfJBPltEoCEÓaív¿¿xUYéZf¡1c4QA“)F¿oaéINÓ.ÉxOín:O\ra\nÉOBbDexH(ñ“CáIVíMT,,YHTNXGjEBò¿,lUXe¡ O?UtüUsCm—¿;Oxq\nlA”\ne¡…osxYFñ(JósDJrpñPG!ñ:lE,ltP”\r¡Q;”¿tY”u)Iá—L1B¡J?jEñ;ÉRma—¿BMñz12éScfüCV —gaj”,JájTú'


In [36]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
def accuracy(labels, logits):
    return tf.keras.metrics.sparse_categorical_accuracy(labels, logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
example_batch_acc  = accuracy(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Loss:      ", example_batch_loss.numpy().mean())
print("Accuracy:      ", example_batch_acc.numpy().mean())

Prediction shape:  (64, 200, 80)  # (batch_size, sequence_length, vocab_size)
Loss:       4.38114
Accuracy:       0.045078125


In [37]:
optimizer = tf.keras.optimizers.Adam() 
model.compile(optimizer=optimizer, loss=loss)

In [38]:
patience = 10
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

In [39]:
checkpoint_dir = './checkpoints'+ datetime.datetime.now().strftime("_%Y.%m.%d-%H:%M:%S")
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(tr_dataset, epochs=epochs, callbacks=[checkpoint_callback, early_stop] , validation_data=val_dataset)
print ("Training stopped as there was no improvement after {} epochs".format(patience))

Epoch 1/70
1/1 [==============================] - 5s 5s/step - loss: 4.3811 - val_loss: 4.2922
Epoch 2/70
1/1 [==============================] - 2s 2s/step - loss: 4.2906 - val_loss: 4.6582
Epoch 3/70
1/1 [==============================] - 2s 2s/step - loss: 4.6700 - val_loss: 3.9277
Epoch 4/70
1/1 [==============================] - 2s 2s/step - loss: 3.9150 - val_loss: 3.8446
Epoch 5/70
1/1 [==============================] - 2s 2s/step - loss: 3.8431 - val_loss: 3.7250
Epoch 6/70
1/1 [==============================] - 2s 2s/step - loss: 3.7183 - val_loss: 3.5047
Epoch 7/70
1/1 [==============================] - 2s 2s/step - loss: 3.4921 - val_loss: 3.3046
Epoch 8/70
1/1 [==============================] - 2s 2s/step - loss: 3.2899 - val_loss: 3.2089
Epoch 9/70
1/1 [==============================] - 2s 2s/step - loss: 3.1887 - val_loss: 3.1905
Epoch 10/70
1/1 [==============================] - 2s 2s/step - loss: 3.1526 - val_loss: 3.2084
Epoch 11/70
1/1 [==============================] 